<a href="https://colab.research.google.com/github/UmeshGayashan/Software_Marks_Giving/blob/main/Properly_Submission_Files.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Mount Google Drive

In [2]:
import os
from glob import glob
from bs4 import BeautifulSoup

from google.colab import drive
drive.mount('/content/drive')

DATASET_PATH = '/content/drive/MyDrive/SS_Dataset'

Mounted at /content/drive


## 2. Extract problem statements from Problems/*.html

In [3]:
def extract_problem_text(html_file):
    with open(html_file, 'r', encoding='utf-8') as f:
        soup = BeautifulSoup(f, 'html.parser')
        text = soup.get_text(separator="\n", strip=True)
        # Extract first 150 words
        words = text.split()
        return ' '.join(words[:150])

problem_texts = {}  # {problem_id: first150words}
problems_dir = os.path.join(DATASET_PATH, "Problems")
html_files = glob(os.path.join(problems_dir, "p*.html"))

for html_file in html_files:
    problem_id = os.path.splitext(os.path.basename(html_file))[0]   # 'p00000'
    problem_texts[problem_id] = extract_problem_text(html_file)

print(f"Extracted {len(problem_texts)} problem statements.")

Extracted 10 problem statements.


## 3. Gather up to 3 .c files for every problem

In [4]:
samples_per_problem = 3
all_samples = {}  # {problem_id: [file1, file2, file3] }

problem_dirs = [d for d in os.listdir(DATASET_PATH) if d.startswith('p') and os.path.isdir(os.path.join(DATASET_PATH, d))]
for pid in problem_dirs:
    c_folder = os.path.join(DATASET_PATH, pid, 'C')
    if os.path.isdir(c_folder):
        c_files = sorted(glob(os.path.join(c_folder, '*.c')))
        all_samples[pid] = c_files[:samples_per_problem]

# Print the samples with the first 150 words of the problem
for pid in sorted(all_samples.keys()):
    print(f"\nProblem: {pid}")
    # Print first 150 words (if available)
    problem_preview = problem_texts.get(pid, "[No statement found]")
    print(f"Statement (first 150 words):\n{problem_preview}\n")
    for i, fpath in enumerate(all_samples[pid], 1):
        print(f"  Sample {i}: {fpath}")



Problem: p00000
Statement (first 150 words):
QQ Write a program which prints multiplication tables in the following format: 1x1=1 1x2=2 . . 9x8=72 9x9=81 Input No input. Output 1x1=1 1x2=2 . . 9x8=72 9x9=81 Template for C #include<stdio.h> int main(){ return 0; } Template for C++ #include<iostream> using namespace std; int main(){ return 0; } Template for Java class Main{ public static void main(String[] a){ } }

  Sample 1: /content/drive/MyDrive/SS_Dataset/p00000/C/s000369988.c
  Sample 2: /content/drive/MyDrive/SS_Dataset/p00000/C/s000552118.c
  Sample 3: /content/drive/MyDrive/SS_Dataset/p00000/C/s000899239.c

Problem: p00001
Statement (first 150 words):
List of Top 3 Hills There is a data which provides heights (in meter) of mountains. The data is only for ten mountains. Write a program which prints heights of the top three mountains in descending order. Input Height of mountain 1 Height of mountain 2 Height of mountain 3 . . Height of mountain 10 Constraints 0 ≤ height of mounta